In [ ]:
import os
import requests
from dotenv import load_dotenv
import time
import json
import pandas as pd
from typing import Dict, List

In [19]:
load_dotenv()

api_key = os.getenv('ETHERSCAN_API_KEY')  
coin_key = os.getenv('GECKO_API')

if not api_key:
    raise ValueError("Please set the ETHERSCAN_API_KEY environment variable")

if not coin_key:
    raise ValueError("Please set the GECKO_API environment variable")

print("Both API keys loaded successfully")

Both API keys loaded successfully


In [20]:
class EtherscanAPI:
    def __init__(self, api_key: str, coin_key: str):
        self.api_key = api_key
        self.coin_key = coin_key
        self.base_url = "https://api.etherscan.io/v2/api"

    def get_transaction_details(self, tx_hash: str):
        """Fetch raw transaction details (from, to, value, gasPrice, hash, nonce, input)"""
        url = (
            f"{self.base_url}"
            f"?chainid=1"
            f"&module=proxy"
            f"&action=eth_getTransactionByHash"
            f"&txhash={tx_hash}"
            f"&apikey={self.api_key}"
        )
        response = requests.get(url).json()
        return response.get("result", {})

    def get_transaction_receipt(self, tx_hash: str):
        """Fetch transaction receipt (logs, status, gasUsed, contractAddress, blockNumber)"""
        url = (
            f"{self.base_url}"
            f"?chainid=1"
            f"&module=proxy"
            f"&action=eth_getTransactionReceipt"
            f"&txhash={tx_hash}"
            f"&apikey={self.api_key}"
        )
        response = requests.get(url).json()
        return response.get("result", {})

    def get_token_info(self, contract_address: str):
        """Fetch ERC20 token metadata using CoinGecko"""
        try:
            url = f"https://api.coingecko.com/api/v3/coins/ethereum/contract/{contract_address}"
            response = requests.get(url).json()

            if "error" in response:
                return {}

            return {
                "name": response.get("name"),
                "symbol": response.get("symbol"),
                "decimals": response.get("detail_platforms", {})
                               .get("ethereum", {})
                               .get("decimal_place", 18)
            }
        except Exception as e:
            print(f"Error fetching token info from CoinGecko: {e}")
            return {"name": "Unknown Token", "symbol": "UNKNOWN", "decimals": 18}

    def get_single_transaction_analysis(self, tx_hash: str):
        """Analyze a single transaction: ETH transfer, ERC20 transfer, approval, or contract interaction"""
        try:
            tx_details = self.get_transaction_details(tx_hash)
            tx_receipt = self.get_transaction_receipt(tx_hash)

            if not tx_details or not tx_receipt:
                return None

            # Get block timestamp
            block_number = tx_receipt.get("blockNumber", "0x0")
            block_url = (
                f"{self.base_url}"
                f"?chainid=1"
                f"&module=proxy"
                f"&action=eth_getBlockByNumber"
                f"&tag={block_number}"
                f"&boolean=true"
                f"&apikey={self.api_key}"
            )
            block_response = requests.get(block_url).json()
            block_data = block_response.get("result", {})
            timestamp = str(int(block_data["timestamp"], 16)) if block_data.get("timestamp") else "0"

            # Extract logs
            logs = tx_receipt.get("logs", [])

            # Topics
            transfer_topic = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
            approval_topic = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"

            # Default ETH value
            eth_value = int(tx_details.get("value", "0x0"), 16) / 1e18

            tx_data = {
                "hash": tx_hash,
                "from": tx_details.get("from", ""),
                "to": tx_details.get("to", ""),
                "blockNumber": block_number,
                "timeStamp": timestamp,
                "gasUsed": tx_receipt.get("gasUsed", "0"),
                "gasPrice": tx_details.get("gasPrice", "0"),
                "status": tx_receipt.get("status", "0x0"),
                # Defaults to prevent KeyError
                "value": "0",
                "tokenName": None,
                "tokenSymbol": None,
                "tokenDecimal": None,
                "tokenAddress": None,
                "tx_type": "unknown"
            }

            # ERC20 Transfer
            for log in logs:
                topics = log.get("topics", [])
                if topics and topics[0] == transfer_topic and len(topics) >= 3:
                    from_address = "0x" + topics[1][-40:]
                    to_address = "0x" + topics[2][-40:]
                    token_contract = log.get("address", "")

                    token_info = self.get_token_info(token_contract)
                    data = log.get("data", "0x0")

                    try:
                        amount_wei = int(data, 16)
                        decimals = int(token_info.get("decimals", 18))
                        amount = amount_wei / (10**decimals)
                    except:
                        amount, decimals = 0, 18

                    tx_data.update({
                        "from": from_address,
                        "to": to_address,
                        "value": str(amount),
                        "tokenName": token_info.get("name"),
                        "tokenSymbol": token_info.get("symbol"),
                        "tokenDecimal": str(decimals),
                        "tokenAddress": token_contract,
                        "tx_type": "erc20_transfer"
                    })
                    return tx_data  # first match only

            # ERC20 Approval
            for log in logs:
                topics = log.get("topics", [])
                if topics and topics[0] == approval_topic and len(topics) >= 3:
                    owner = "0x" + topics[1][-40:]
                    spender = "0x" + topics[2][-40:]
                    token_contract = log.get("address", "")

                    token_info = self.get_token_info(token_contract)
                    data = log.get("data", "0x0")

                    try:
                        amount_wei = int(data, 16)
                        decimals = int(token_info.get("decimals", 18))
                        amount = amount_wei / (10**decimals)
                    except:
                        amount, decimals = 0, 18

                    tx_data.update({
                        "owner": owner,
                        "spender": spender,
                        "value": str(amount),
                        "tokenName": token_info.get("name"),
                        "tokenSymbol": token_info.get("symbol"),
                        "tokenDecimal": str(decimals),
                        "tokenAddress": token_contract,
                        "tx_type": "erc20_approval"
                    })
                    return tx_data

            # ETH Transfer
            if eth_value > 0: # Ensures ETH was transferred (not just a contract call)
                tx_data.update({
                    "value": str(eth_value),
                    "tokenName": "Ethereum",
                    "tokenSymbol": "ETH",
                    "tokenDecimal": "18",
                    "tokenAddress": "0x0000000000000000000000000000000000000000",
                    "tx_type": "eth_transfer"
                })
                return tx_data

            # Contract Interaction
            if tx_details.get("input") and tx_details["input"] != "0x":
                method_id = tx_details["input"][:10]
                tx_data.update({
                    "methodId": method_id,
                    "input": tx_details["input"],
                    "tx_type": "contract_interaction"
                })
                return tx_data

            # Default (failed/other)
            tx_data.update({"tx_type": "other"})
            return tx_data

        except Exception as e:
            print(f"Error analyzing transaction {tx_hash}: {e}")
            return None

In [7]:
# Gets transaction details

url = (
    f"https://api.etherscan.io/v2/api"
    f"?chainid=1"
    f"&module=proxy"
    f"&action=eth_getTransactionByHash"
    f"&txhash=0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64"
    f"&apikey={api_key}"
)

response = requests.get(url)
data = response.json()
data

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'blockHash': '0xda703d76f51451c91690baeb9f6bb6f75e40c3e5ea445d3ea1f2cc547c8b5dc5',
  'blockNumber': '0x164fec5',
  'from': '0x000000002a634f97eef66251b538046514d94f4a',
  'gas': '0x310a8',
  'gasPrice': '0x1193fcf4',
  'maxFeePerGas': '0x1193fcf4',
  'maxPriorityFeePerGas': '0x1193fcf4',
  'hash': '0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64',
  'input': '0x5400580000000000000000000e2f9abfb000000100000088000bb800003c000001000000000000017a9ae6dcd0fdb1bdb90074332a24318d56f9cca677a242aff668314492bf8010ee9f68ee4e4d311e854ae14c53f5b25a917f8536e55efaf41424332a24318d56f9cca677a242aff668314492bf809a',
  'nonce': '0xbf97',
  'to': '0xa12ffb73efa3b5f2edb0c528cbff7aee34d60e37',
  'transactionIndex': '0x121',
  'value': '0xfec5',
  'type': '0x2',
  'accessList': [{'address': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    'storageKeys': ['0x1ff39f507ef774da8fa14a93b0aba836b6b7e33794eea2799447d1a1596c75e2',
     '0x373f3dedd0fc6641dc

In [ ]:
# Get_transaction_receipt

url = (
    f"https://api.etherscan.io/v2/api"
    f"?chainid=1"
    f"&module=proxy"
    f"&action=eth_getTransactionReceipt"
    f"&txhash=0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64"
    f"&apikey={api_key}"
)

response = requests.get(url)
data = response.json()
data

{'jsonrpc': '2.0',
 'id': 1,
 'result': {'blockHash': '0xda703d76f51451c91690baeb9f6bb6f75e40c3e5ea445d3ea1f2cc547c8b5dc5',
  'blockNumber': '0x164fec5',
  'contractAddress': None,
  'cumulativeGasUsed': '0x1688aec',
  'effectiveGasPrice': '0x1193fcf4',
  'from': '0x000000002a634f97eef66251b538046514d94f4a',
  'gasUsed': '0x22543',
  'logs': [{'address': '0x000000000004444c5dc75cb358380d2e3de08a90',
    'topics': ['0x40e9cecb9f5f1f1c5b9c97dec2917b7ee92e57ba5563708daca94dd84ad7112f',
     '0x42e23f58b04d83e02d6c8fcd38775e3c4fa0bb65bad8938e3dd207c33ef93451',
     '0x000000000000000000000000a12ffb73efa3b5f2edb0c528cbff7aee34d60e37'],
    'data': '0xfffffffffffffffffffffffffffffffffffffffffffffffffff1d0654050000000000000000000000000000000000000000000000000017a9ae6dcd0fdb1bdb90000000000000000000000000000000000000525a1734fcdc65b42e6ede3757d00000000000000000000000000000000000000000000001b6ecb736f37f301f5000000000000000000000000000000000000000000000000000000000002313e00000000000000000000000000

In [4]:
import json
import numpy as np
import pandas as pd

# Helper to clean up objects that json can’t handle
def clean_for_json(obj):
    if isinstance(obj, (np.bool_, np.int64, np.float64)):
        return obj.item()
    if isinstance(obj, (pd.Timestamp,)):
        return obj.isoformat()
    return str(obj)

# Your transaction hash
tx_hash = "0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64"

# Run the tracker
result = tracker.analyze_transactions(tx_hash, is_transaction_hash=True)

# Print JSON safely
print(json.dumps(result, indent=2, default=clean_for_json))


Detected transaction hash, analyzing single transaction: 0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64
Analyzing single transaction: 0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64
{
  "analysis_type": "single_transaction_analysis",
  "transaction_hash": "0xb505ed73fe278a8dcaa1eb7513b2457f5db3ee0392de852e374fd26a6833df64",
  "transaction_details": {
    "from": "0x000000000004444c5dc75cb358380d2e3de08a90",
    "to": "0x332a24318d56f9cca677a242aff668314492bf80",
    "value": 6984.031111378862,
    "token_name": "Unknown Token",
    "token_symbol": "UNKNOWN",
    "token_address": "0x10ee9f68ee4e4d311e854ae14c53f5b25a917f85",
    "gas_fee_eth": 4.146776155542e-05,
    "timestamp": "2025-09-19T08:49:35",
    "block_number": "0x164fec5",
    "transaction_status": "failed"
  },
  "anomaly_analysis": {
    "gas_fee_category": "normal",
    "transfer_amount_category": "small_active",
    "is_high_gas_anomaly": false,
    "is_large_transfer": false,
    "

In [16]:
import requests
import time

class EtherscanAPI:
    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "https://api.etherscan.io/v2/api"
    
    def get_token_transactions(self, address: str, max_pages: int = 10, offset: int = 1000, 
                               sort: str = 'desc', start_block: int = 0, end_block: str = 'latest'):
        """Get token transactions for an address"""
        token_txs = []
        
        for page in range(1, max_pages + 1):
            url = (
                f"{self.base_url}"
                f"?chainid=1"
                f"&module=account"
                f"&action=tokentx"
                f"&address={address}"
                f"&startblock={start_block}"
                f"&endblock={end_block}"
                f"&page={page}"
                f"&offset={offset}"
                f"&sort={sort}"
                f"&apikey={self.api_key}"
            )

            response = requests.get(url).json()
            txs = response.get("result", [])
            
            if not txs:  # Stop if no more results
                break

            token_txs.extend(txs)
            print(f"Fetched page {page}, got {len(txs)} transactions")

            time.sleep(0.2)  # Helps with API rate limits

        return token_txs

In [17]:
# Step 1: Create an instance of the class
usdt_tx = EtherscanAPI(API_KEY)

# Step 2: Use the instance to call the method
data = usdt_tx.get_token_transactions("0xdAC17F958D2ee523a2206206994597C13D831ec7")
data

Fetched page 1, got 1000 transactions
Fetched page 2, got 1000 transactions
Fetched page 3, got 1000 transactions
Fetched page 4, got 1000 transactions
Fetched page 5, got 1000 transactions
Fetched page 6, got 1000 transactions
Fetched page 7, got 1000 transactions
Fetched page 8, got 1000 transactions
Fetched page 9, got 1000 transactions
Fetched page 10, got 1000 transactions


[{'blockNumber': '23389879',
  'timeStamp': '1758197495',
  'hash': '0xb34b546a138c3c5edc05f7cba17b91bc814176816d63965ad526e2de2425ed8d',
  'nonce': '153830',
  'blockHash': '0xf336df55b3093080371807d6a2ef6f1c977719a7291f652067bbcae7e4ff3460',
  'from': '0x18e296053cbdf986196903e889b7dca7a73882f6',
  'contractAddress': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'to': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'value': '23725800',
  'tokenName': 'Tether USD',
  'tokenSymbol': 'USDT',
  'tokenDecimal': '6',
  'transactionIndex': '61',
  'gas': '90000',
  'gasPrice': '6346505465',
  'gasUsed': '46109',
  'cumulativeGasUsed': '7799820',
  'input': 'deprecated',
  'methodId': '0xa9059cbb',
  'functionName': 'transfer(address _to, uint256 _value)',
  'confirmations': '427'},
 {'blockNumber': '23389693',
  'timeStamp': '1758195239',
  'hash': '0xe397fdbc21b3c804820333d0a52855d866c8472b730086ef5c9e87ce801b6cf0',
  'nonce': '1390',
  'blockHash': '0x723fc71174fc7f26d2fcf92e0dd021e4f96

In [20]:
# Load raw data into DataFrame
df_usdt = pd.DataFrame(data)

# Convert timestamp (unix → datetime)
df_usdt["timeStamp"] = pd.to_datetime(df_usdt["timeStamp"].astype(int), unit="s")

# Convert value to human-readable using tokenDecimal
df_usdt["value"] = df_usdt.apply(
    lambda row: int(row["value"]) / (10 ** int(row["tokenDecimal"])),
    axis=1
)

# Convert gas fee to ETH
df_usdt["gasFee_ETH"] = (
    df_usdt["gasUsed"].astype(int) * df_usdt["gasPrice"].astype(int) / 1e18
)

# Show last few rows
df_usdt.tail()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,methodId,functionName,confirmations,gasFee_ETH
9995,17829995,2023-08-02 21:27:23,0x6c6738eaf1cec4b273dc4fa89d4c2c2c7d28b809024f...,17,0x6fb40314065a2b5443fc930041d34d992e3a6f59ba0a...,0xf5ef7a8db55a0a486ba78609a7181c9b3be8985f,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdac17f958d2ee523a2206206994597c13d831ec7,0.000000,Tether USD,...,84,61056,26112092486,40395,10587520,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",5560321,0.001055
9996,17829467,2023-08-02 19:41:35,0x31ecf69d627fe941a3f43f2ffa72dc17696cec4db4c4...,26,0x4d447653ca6bb8802754c2f8fb9fb60aaf5e0e35888d...,0x2743b02ff450f23b7ee9e2691f74018af9fb1510,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdac17f958d2ee523a2206206994597c13d831ec7,10.000000,Tether USD,...,96,55316,26136357533,46097,8926928,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",5560849,0.001205
9997,17828320,2023-08-02 15:51:35,0x4df6981fdbd1bab845e5dd233f427bc3d5251d59bad6...,41,0xa21171777b553a6c3fe3280894d49da829aa6a519124...,0x7905d15167c0ee907cbb25e6acddcd0d98d8099b,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdac17f958d2ee523a2206206994597c13d831ec7,200.000000,Tether USD,...,46,70000,34399991544,46097,6297993,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",5561996,0.001586
9998,17827164,2023-08-02 11:58:35,0x566cc7c9321eef3f9be97b0f0d8fe91207745c961d92...,6,0xe3b40ac5ae1c8e0e1ff968e5ab39679f2e852f81ffc8...,0x240d96179852e69ae9e09867d1c5fb3b7e6b3923,0x50078f0412bcae978bc1bc455dc418898db2107d,0xdac17f958d2ee523a2206206994597c13d831ec7,10.000000,UNIFI,...,75,60000,21610302592,51448,6921741,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",5563152,0.001112
9999,17826921,2023-08-02 11:09:11,0x297d87a44f72294ac15b91b2d0130f9b947b1bc9cc97...,674900,0x4142b880b7266d17696fca9d707559de6f5346b24227...,0x3c02290922a3618a4646e3bbca65853ea45fe7c6,0xdac17f958d2ee523a2206206994597c13d831ec7,0xdac17f958d2ee523a2206206994597c13d831ec7,37.801277,Tether USD,...,14,90000,16655384182,46109,2165051,deprecated,0xa9059cbb,"transfer(address _to, uint256 _value)",5563395,0.000768


In [21]:
df_usdt['gasFee_ETH'].agg(['max', 'min', 'mean'])

max     0.859339
min     0.000005
mean    0.001301
Name: gasFee_ETH, dtype: float64

In [ ]:
# Extract just the date (drop time)
df_usdt['date'] = df_usdt['timeStamp'].dt.date

# Group by date and calculate max, min, avg transfer amounts
daily_stats = df_usdt.groupby('date')['value'].agg(
    largest_transfer='max',
    smallest_transfer='min',
    average_transfer='mean'
).reset_index()

daily_stats

,date,largest_transfer,smallest_transfer,average_transfer
0,2023-08-02,1.242976e+16,0.000000,2.095996e+15
1,2023-08-03,2.016574e+04,10.000000,3.310539e+03
2,2023-08-04,2.250000e+09,10.000000,4.500003e+08
3,2023-08-06,2.552880e+01,0.000000,8.509600e+00
4,2023-08-07,3.000000e+03,0.000000,5.907900e+02
...,...,...,...,...
773,2025-09-14,3.500000e+03,10.500000,5.764219e+02
774,2025-09-15,1.131169e+03,0.000000,8.926573e+01
775,2025-09-16,4.516776e+02,0.000000,5.950398e+01
776,2025-09-17,9.990000e+04,0.499769,5.614740e+03


In [37]:
# Choose a window (e.g., hourly)
df_usdt['hour'] = df_usdt['timeStamp'].dt.floor('H')

C:\Users\FFFO CASHIER PT\AppData\Local\Temp\ipykernel_15768\657437451.py:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_usdt['hour'] = df_usdt['timeStamp'].dt.floor('H')


In [42]:
# Analyze sender behavior (number of transactions per hour)

# Count transactions per sender per hour
sender_counts = df_usdt.groupby(['from', 'hour'])['hash'].count().rename("tx_count").reset_index()

# Compute mean and std per sender
def detect_sender_anomalies(group):
    mean = group['tx_count'].mean()
    std = group['tx_count'].std()
    # Flag anomalous hours: tx_count > mean + 2*std
    return group[group['tx_count'] > mean + 2*std]

sender_anomalies = sender_counts.groupby('from').apply(detect_sender_anomalies).reset_index(drop=True)
sender_anomalies

C:\Users\FFFO CASHIER PT\AppData\Local\Temp\ipykernel_15768\2298875931.py:13: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sender_anomalies = sender_counts.groupby('from').apply(detect_sender_anomalies).reset_index(drop=True)


,from,hour,tx_count
0,0x0000000000000000000000000000000000000000,2025-04-27 22:00:00,5
1,0x02261614eadc4afb6623e189ae5e1bb3eec2d341,2024-08-06 12:00:00,17
2,0x02f934c89fb93dfb5be3bac7198cab4cfa675d37,2024-02-02 01:00:00,5
3,0x18e296053cbdf986196903e889b7dca7a73882f6,2025-07-13 23:00:00,2
4,0x18e296053cbdf986196903e889b7dca7a73882f6,2025-07-22 10:00:00,2
...,...,...,...
72,0xf89d7b9c864f589bbf53a82105107622b35eaa40,2025-01-20 02:00:00,2
73,0xf89d7b9c864f589bbf53a82105107622b35eaa40,2025-01-20 06:00:00,2
74,0xf89d7b9c864f589bbf53a82105107622b35eaa40,2025-01-20 12:00:00,2
75,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,2023-12-28 10:00:00,2


In [46]:
# Convert raw value to human-readable
df_usdt['human_value'] = df_usdt.apply(
    lambda row: int(row['value']) / (10 ** int(row['tokenDecimal'])), axis=1
)

# Aggregate total received per recipient per day
df_usdt['date'] = df_usdt['timeStamp'].dt.date
recipient_daily = df_usdt.groupby(['to', 'date'])['human_value'].sum().reset_index()

# Detect anomalies
def detect_recipient_anomalies(group):
    median = group['human_value'].median()
    std = group['human_value'].std()
    return group[group['human_value'] > median + 2*std]

recipient_anomalies = recipient_daily.groupby('to', group_keys=False).apply(detect_recipient_anomalies)
recipient_anomalies

c:\MY_WORK\On-chain_Anomaly_Tracker\anomaly\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: overflow encountered in square
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\FFFO CASHIER PT\AppData\Local\Temp\ipykernel_15768\261306317.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recipient_anomalies = recipient_daily.groupby('to', group_keys=False).apply(detect_recipient_anomalies)


,to,date,human_value
18,0x036888ca894efddf8efea831d8fa83a0365d073a,2025-08-28,5.000000e+11
34,0x0802dd2edd3f9fad39a9173b4595be819f201d61,2024-01-23,1.260000e+13
40,0x0802dd2edd3f9fad39a9173b4595be819f201d61,2024-03-05,1.600000e+13
54,0x0802dd2edd3f9fad39a9173b4595be819f201d61,2024-06-26,1.500000e+13
172,0x26352d20e6a05e04a1ecc75d4a43ae9989272621,2025-08-29,2.000000e+12
173,0x26352d20e6a05e04a1ecc75d4a43ae9989272621,2025-09-11,2.000000e+12
200,0x2c926e3a2f8b54e74aeaf7e143100e3b783a2c06,2025-07-02,3.000010e+12
266,0x48a571478930c5a2a1432f3400a9708f54fd85bb,2024-07-03,2.000000e+06
413,0x82bc4ea1f276034acc57c1f594c6684a7b4209fc,2023-10-26,3.190000e+10
422,0x82bc4ea1f276034acc57c1f594c6684a7b4209fc,2024-02-05,2.600000e+10


In [36]:
# Only compute z-score for addresses with enough variation

from scipy.stats import zscore

def safe_zscore(x):
    if x.std(ddof=0) == 0:
        return [0]*len(x)  # all identical values => z-score = 0
    else:
        return zscore(x, ddof=0)

tx_counts['z_score'] = tx_counts.groupby('from')['tx_count'].transform(safe_zscore)
tx_counts

,from,hour,tx_count,z_score
0,0x0000000000000000000000000000000000000000,2023-10-05 12:00:00,1,-0.364405
1,0x0000000000000000000000000000000000000000,2024-09-28 20:00:00,1,-0.364405
2,0x0000000000000000000000000000000000000000,2024-11-16 09:00:00,1,-0.364405
3,0x0000000000000000000000000000000000000000,2025-02-02 06:00:00,1,-0.364405
4,0x0000000000000000000000000000000000000000,2025-02-10 18:00:00,1,-0.364405
...,...,...,...,...
8279,0xffec0067f5a79cff07527f63d83dd5462ccf8ba4,2024-03-02 02:00:00,1,0.000000
8280,0xffecc4533afb260ae7b0cfae3daf352167346548,2024-02-25 22:00:00,1,0.000000
8281,0xfff46b7f11dad4c64bc83c9663cd0cd26b34a444,2024-12-28 23:00:00,1,0.000000
8282,0xfff5ebaef7c40bc15708b21023481d7580054a3a,2024-11-23 14:00:00,1,0.000000


In [3]:
address = "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"  # USDC

url = (
      f"https://api.etherscan.io/v2/api"
      f"?chainid=1"
      f"&module=account"
      f"&action=tokentx"
      f"&address={address}"
      f"&startblock=0"
      f"&endblock=latest"
      f"&page=1"
      f"&offset=10"
      f"&sort=desc"
      f"&apikey={API_KEY}"
    )

response = requests.get(url)
data = response.json()
data

{'status': '1',
 'message': 'OK',
 'result': [{'blockNumber': '23380615',
   'timeStamp': '1758085763',
   'hash': '0x98e175b0ed854cb1d799e04a06ff4b7055ca808baefa8288fb4a7e0db85df963',
   'nonce': '2',
   'blockHash': '0x9482601d789e2fb939431f6c1dd197985ae740727a26fa6837ae0b8342f6fcaa',
   'from': '0xf436c3f1a034fc6e19fc0a6cca92b4eceef1f7b0',
   'contractAddress': '0xdac17f958d2ee523a2206206994597c13d831ec7',
   'to': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
   'value': '4000000',
   'tokenName': 'Tether USD',
   'tokenSymbol': 'USDT',
   'tokenDecimal': '6',
   'transactionIndex': '98',
   'gas': '59942',
   'gasPrice': '208458004',
   'gasUsed': '46085',
   'cumulativeGasUsed': '12302800',
   'input': 'deprecated',
   'methodId': '0xa9059cbb',
   'functionName': 'transfer(address _to, uint256 _value)',
   'confirmations': '9409'},
  {'blockNumber': '23380569',
   'timeStamp': '1758085211',
   'hash': '0xe26aa8ddd85946065db1c1c73692692127e9b66f5bec2c6a1cced2d0e4a52d21',
   'nonce

In [ ]:
import pandas as pd

# Example: let's assume df is your DataFrame with all the transactions

def get_transaction_by_hash(df: pd.DataFrame, tx_hash: str) -> pd.DataFrame:
    """
    Retrieve transaction details from a DataFrame using the transaction hash.
    
    Args:
        df (pd.DataFrame): DataFrame containing transaction data
        tx_hash (str): Transaction hash
    
    Returns:
        pd.DataFrame: Filtered DataFrame containing the transaction
    """
    return df[df["hash"] == tx_hash]


# Example usage
tx_hash = "0x6c6738eaf1cec4b273dc4fa89d4c2c2c7d28b809024f..."
result = get_transaction_by_hash(df, tx_hash)

print(result)


In [ ]:
This will return the row(s) where the hash column matches your transaction hash.
If you want just a dictionary instead of a DataFrame row, you can do:

def get_transaction_by_hash_dict(df: pd.DataFrame, tx_hash: str) -> dict:
    row = df[df["hash"] == tx_hash]
    if row.empty:
        return {}
    return row.to_dict(orient="records")[0]  # first matching record


In [16]:
import requests

contract_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"  # USDT contract
url = f"https://pro-api.coingecko.com/api/v3/coins/ethereum/contract/{contract_address}"

headers = {"x-cg-pro-api-key": coin_key}

response = requests.get(url, headers=headers)

print(response.status_code)
response.json()

200


{'id': 'tether',
 'symbol': 'usdt',
 'name': 'Tether',
 'web_slug': 'tether',
 'asset_platform_id': 'ethereum',
 'platforms': {'ethereum': '0xdac17f958d2ee523a2206206994597c13d831ec7',
  'klay-token': '0xd077a400968890eacc75cdc901f0356c943e4fdb',
  'tron': 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t',
  'terra-2': 'ibc/9B19062D46CAB50361CE9B0A3E6D0A7A53AC9E7CB361F32A73CC733144A9A9E5',
  'aptos': '0x357b0b74bc833e95a115ad22604854d6b0fca151cecd94111770e5d6ffc9dc2b',
  'avalanche': '0x9702230a8ea53601f5cd2dc00fdbc13d4df4a8c7',
  'solana': 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB',
  'near-protocol': 'usdt.tether-token.near',
  'kava': '0x919c1c267bc06a7039e03fcc2ef738525769109c',
  'celo': '0x48065fbbe25f71c9282ddf5e1cd6d6a887483d5e',
  'the-open-network': 'EQCxE6mUtQJKFnGfaROTKOt1lZbDiiX1kCixRv7Nw2Id_sDs'},
 'detail_platforms': {'ethereum': {'decimal_place': 6,
   'contract_address': '0xdac17f958d2ee523a2206206994597c13d831ec7',
   'geckoterminal_url': 'https://www.geckoterminal.com/eth/tok